# 🛡️ Credit Risk Pro Model Eğitimi

Bu notebook, **XGBoost Classifier** kullanarak detaylı kredi risk tahmini için optimize edilmiş bir model eğitmektedir.

## 📋 Model Özellikleri

- **Model Tipi**: XGBoost (Extreme Gradient Boosting) Classifier
- **Değişken Sayısı**: 16 değişken (13 temel + 3 türetilmiş)
- **Kullanım Amacı**: Detaylı risk analizi için optimize edilmiş model
- **Optimizasyon**: RandomizedSearchCV ile hiperparametre optimizasyonu


In [ ]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CREDIT RISK PRO MODEL EĞİTİMİ")
print("=" * 80)
print()


## 📂 2. Veri Yükleme

Bu adımda, Lending Club veri setini yüklüyoruz. Bu veri seti gerçek kredi verilerini içermektedir.

**Veri Seti**: `lending_club_cleaned.csv`
- Toplam kayıt sayısını ve sütun sayısını kontrol ediyoruz


In [ ]:
# Veri dosyasını yükle
INPUT_FILE = 'lending_club_cleaned.csv'
df = pd.read_csv(INPUT_FILE)

print(f"Toplam kayıt: {len(df)}")
print(f"Sütun sayısı: {len(df.columns)}")
print(f"\nİlk 5 satır:")
df.head()


## 🔧 3. Özellik Mühendisliği (Feature Engineering)

Bu bölümde, model performansını artırmak için yeni özellikler türetiyoruz:

### Türetilmiş Özellikler:
- **loan_to_income**: Kredi tutarı / Yıllık gelir oranı
  - Bu oran, müşterinin kredi yükümlülüğünün gelirine göre ne kadar büyük olduğunu gösterir
  
- **installment_to_income**: Aylık taksit / Aylık gelir oranı (PTI - Payment-to-Income)
  - Müşterinin aylık gelirinin ne kadarını kredi taksitine ayırdığını gösterir
  - Yüksek PTI değerleri, finansal sıkışıklık sinyali verebilir
  
- **balance_income_ratio**: Döner kredi bakiyesi / Yıllık gelir oranı
  - Müşterinin döner kredi borcunun gelirine göre ne kadar büyük olduğunu gösterir

### Seçilen Özellikler (16 değişken):
1. **loan_amnt**: Kredi tutarı
2. **annual_inc**: Yıllık gelir
3. **installment**: Aylık taksit
4. **int_rate**: Faiz oranı
5. **dti**: Debt-to-Income ratio (Borç/Gelir oranı)
6. **fico_range_low**: FICO kredi skoru (düşük)
7. **fico_range_high**: FICO kredi skoru (yüksek)
8. **revol_bal**: Döner kredi bakiyesi
9. **revol_util**: Döner kredi kullanım oranı
10. **total_acc**: Toplam hesap sayısı
11. **open_acc**: Açık hesap sayısı
12. **pub_rec**: Kamu kayıtları
13. **inq_last_6mths**: Son 6 ayda sorgulama sayısı
14. **loan_to_income**: Türetilmiş özellik
15. **installment_to_income**: Türetilmiş özellik (PTI)
16. **balance_income_ratio**: Türetilmiş özellik

### Kategorik Değişkenler:
- **home_ownership**: Ev durumu (Kiracı, Ev sahibi, vb.)
- **purpose**: Kredi amacı
- **grade**: Kredi notu (A, B, C, D, E, F, G)
- **emp_length**: İstihdam süresi
- **verification_status**: Gelir doğrulama durumu


In [ ]:
# Türetilmiş özellikler
# 1. loan_to_income: Kredi tutarı / Yıllık gelir
df['loan_to_income'] = df['loan_amnt'] / (df['annual_inc'] + 1)

# 2. installment_to_income: Aylık taksit / Aylık gelir (PTI - Payment-to-Income)
df['installment_to_income'] = df['installment'] / ((df['annual_inc'] / 12) + 1)

# 3. balance_income_ratio: Döner kredi bakiyesi / Yıllık gelir
df['balance_income_ratio'] = df['revol_bal'] / (df['annual_inc'] + 1)

# Pro Model için 16 değişken seçimi (13 temel + 3 türetilmiş)
pro_features = [
    'loan_amnt',              # Kredi tutarı
    'annual_inc',             # Yıllık gelir
    'installment',            # Aylık taksit
    'int_rate',              # Faiz oranı
    'dti',                   # Debt-to-Income ratio
    'fico_range_low',        # FICO skoru (düşük)
    'fico_range_high',       # FICO skoru (yüksek)
    'revol_bal',             # Döner kredi bakiyesi
    'revol_util',            # Döner kredi kullanım oranı
    'total_acc',             # Toplam hesap sayısı
    'open_acc',              # Açık hesap sayısı
    'pub_rec',               # Kamu kayıtları
    'inq_last_6mths',       # Son 6 ayda sorgulama sayısı
    'loan_to_income',        # Türetilmiş: Kredi tutarı / Yıllık gelir
    'installment_to_income', # Türetilmiş: Aylık taksit / Aylık gelir (PTI)
    'balance_income_ratio'  # Türetilmiş: Döner kredi bakiyesi / Yıllık gelir
]

# Kategorik değişkenler (One-Hot Encoding için)
categorical_features = ['home_ownership', 'purpose', 'grade', 'emp_length', 'verification_status']

# Hedef değişken
target = 'loan_status_binary'  # 0: Ödendi, 1: Temerrüt

# Veri hazırlama
X = df[pro_features + categorical_features].copy()
y = df[target].copy()

print(f"Seçilen özellik sayısı: {len(pro_features)} temel + {len(categorical_features)} kategorik")
print(f"\nTüretilmiş özellikler:")
print(f"  - loan_to_income: Kredi tutarı / Yıllık gelir")
print(f"  - installment_to_income: Aylık taksit / Aylık gelir (PTI)")
print(f"  - balance_income_ratio: Döner kredi bakiyesi / Yıllık gelir")
print(f"\nHedef değişken dağılımı:")
print(f"  0 (Ödendi): {(y == 0).sum()} ({(y == 0).sum() / len(y) * 100:.2f}%)")
print(f"  1 (Temerrüt): {(y == 1).sum()} ({(y == 1).sum() / len(y) * 100:.2f}%)")
print(f"\nTüretilmiş özellik istatistikleri:")
print(df[['loan_to_income', 'installment_to_income', 'balance_income_ratio']].describe())


## 🧹 4. Veri Ön İşleme (Preprocessing)

Bu bölümde, eksik değerleri dolduruyoruz ve kategorik değişkenleri sayısal formata dönüştürüyoruz.

### Eksik Değer İşleme:
- **Sayısal değişkenler**: Medyan ile doldurulur (aykırı değerlere daha dayanıklı)
- **Kategorik değişkenler**: Mod (en sık görülen değer) ile doldurulur

### One-Hot Encoding:
- Kategorik değişkenler binary (0/1) sütunlara dönüştürülür
- Her kategori için ayrı bir sütun oluşturulur
- Multicollinearity önlemek için ilk kategori düşürülür (`drop='first'`)


In [ ]:
# Eksik değer kontrolü
print("Eksik değer kontrolü:")
missing_summary = X.isnull().sum()
for col in X.columns:
    missing = missing_summary[col]
    if missing > 0:
        print(f"  {col}: {missing} eksik değer ({missing/len(X)*100:.2f}%)")

# Eksik değerleri doldur
# Sayısal değişkenler için medyan kullanıyoruz (aykırı değerlere daha dayanıklı)
for col in pro_features:
    if X[col].isnull().sum() > 0:
        median_val = X[col].median()
        X[col].fillna(median_val, inplace=True)
        print(f"  {col}: {X[col].isnull().sum()} eksik değer medyan ({median_val:.2f}) ile dolduruldu")

# Kategorik değişkenler için mod (en sık görülen değer) kullanıyoruz
for col in categorical_features:
    if X[col].isnull().sum() > 0:
        mode_val = X[col].mode()[0] if len(X[col].mode()) > 0 else 'Unknown'
        X[col].fillna(mode_val, inplace=True)
        print(f"  {col}: {X[col].isnull().sum()} eksik değer mod ({mode_val}) ile dolduruldu")

print(f"\nToplam eksik değer sayısı: {X.isnull().sum().sum()}")


In [ ]:
# One-Hot Encoding için ColumnTransformer
# ColumnTransformer, farklı sütunlara farklı dönüşümler uygulamamıza olanak sağlar
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', pro_features),  # Sayısal özellikler olduğu gibi geçirilir
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), categorical_features)
        # Kategorik özellikler One-Hot Encoding ile dönüştürülür
        # drop='first': İlk kategoriyi düşür (multicollinearity önleme)
        # handle_unknown='ignore': Eğitim sırasında görülmeyen kategorileri yok say
        # sparse_output=False: Dense array döndür (numpy array)
    ],
    remainder='drop'  # Belirtilmeyen sütunları düşür
)

# Preprocessing uygula
print("Preprocessing uygulanıyor...")
X_processed = preprocessor.fit_transform(X)

# Özellik isimlerini al (One-Hot Encoding sonrası oluşan yeni sütunlar dahil)
feature_names = pro_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

print(f"İşlenmiş özellik sayısı: {X_processed.shape[1]}")
print(f"One-Hot Encoding uygulandı")
print(f"Orijinal özellik sayısı: {len(pro_features) + len(categorical_features)}")
print(f"Yeni özellik sayısı: {X_processed.shape[1]}")


## ✂️ 5. Veri Bölme (Train-Test Split)

Modelin genelleme yeteneğini test etmek için veriyi eğitim ve test setlerine ayırıyoruz:

- **Train Set**: Model eğitimi için kullanılacak (%80)
- **Test Set**: Model performansını değerlendirmek için kullanılacak (%20)
- **Stratify**: Hedef değişkenin dağılımını her iki sette de korur (sınıf dengesizliği için önemli)


In [ ]:
# Veri bölme
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, 
    test_size=0.2,  # %20 test seti
    stratify=y  # Sınıf dağılımını koru
)

print(f"Train set: {X_train.shape[0]} kayıt ({X_train.shape[0]/len(X_processed)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} kayıt ({X_test.shape[0]/len(X_processed)*100:.1f}%)")
print(f"\nTrain set hedef dağılımı:")
print(f"  0 (Ödendi): {(y_train == 0).sum()} ({(y_train == 0).sum() / len(y_train) * 100:.2f}%)")
print(f"  1 (Temerrüt): {(y_train == 1).sum()} ({(y_train == 1).sum() / len(y_train) * 100:.2f}%)")
print(f"\nTest set hedef dağılımı:")
print(f"  0 (Ödendi): {(y_test == 0).sum()} ({(y_test == 0).sum() / len(y_test) * 100:.2f}%)")
print(f"  1 (Temerrüt): {(y_test == 1).sum()} ({(y_test == 1).sum() / len(y_test) * 100:.2f}%)")


## 🎯 6. Model Eğitimi ve Hiperparametre Optimizasyonu

Bu bölümde, XGBoost modelini eğitiyoruz ve en iyi hiperparametreleri bulmak için **RandomizedSearchCV** kullanıyoruz.

### RandomizedSearchCV Nedir?
- GridSearchCV'nin daha hızlı alternatifi
- Belirtilen parametre uzayından **rastgele** kombinasyonlar seçer
- Tüm kombinasyonları denemek yerine, belirli sayıda kombinasyonu test eder
- Daha hızlı sonuç verir, ancak global optimum'u garanti etmez

### Test Edilen Parametreler:
- **n_estimators**: Ağaç sayısı (200, 300, 350, 400, 500)
- **learning_rate**: Öğrenme hızı (0.01, 0.03, 0.05, 0.1)
- **max_depth**: Ağaç derinliği (3, 4, 5, 6)
- **subsample**: Alt örnekleme oranı (0.7, 0.75, 0.8, 0.9)
- **colsample_bytree**: Sütun alt örnekleme (0.7, 0.75, 0.8)
- **min_child_weight**: Minimum çocuk ağırlığı (1, 2, 3)
- **gamma**: Minimum kayıp azaltma (0, 0.1, 0.2)

### Optimizasyon Stratejisi:
- **100 kombinasyon** test edilecek
- **3-fold Cross Validation** ile her kombinasyon değerlendirilecek
- **Scoring metrik**: ROC-AUC (Receiver Operating Characteristic - Area Under Curve)


In [ ]:
# Base model (varsayılan parametrelerle)
base_model = XGBClassifier(
    eval_metric='logloss',  # Binary classification için logloss kullan
    use_label_encoder=False,  # Label encoder kullanma (deprecated)
    n_jobs=-1,  # Tüm CPU çekirdeklerini kullan
    verbosity=0  # Çıktıları gizle
)

# Hiperparametre arama uzayı
param_distributions = {
    'n_estimators': [200, 300, 350, 400, 500],  # Ağaç sayısı
    'learning_rate': [0.01, 0.03, 0.05, 0.1],  # Öğrenme hızı
    'max_depth': [3, 4, 5, 6],  # Ağaç derinliği
    'subsample': [0.7, 0.75, 0.8, 0.9],  # Alt örnekleme oranı
    'colsample_bytree': [0.7, 0.75, 0.8],  # Sütun alt örnekleme
    'min_child_weight': [1, 2, 3],  # Minimum çocuk ağırlığı
    'gamma': [0, 0.1, 0.2]  # Minimum kayıp azaltma
}

print("Hiperparametre optimizasyonu başlatılıyor...")
print("RandomizedSearchCV ile 100 kombinasyon test edilecek (3-fold CV)...")
print("Bu işlem birkaç dakika sürebilir...\n")

# RandomizedSearchCV ile optimizasyon
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=100,  # 100 rastgele kombinasyon test et
    cv=3,  # 3-fold cross validation
    scoring='roc_auc',  # ROC-AUC skorunu optimize et
    n_jobs=-1,  # Paralel işleme
    verbose=1  # İlerleme bilgisi göster
)

# Optimizasyonu çalıştır
random_search.fit(X_train, y_train)

print("\n" + "="*80)
print("OPTİMİZASYON TAMAMLANDI!")
print("="*80)
print(f"\nEn iyi parametreler:")
best_params = random_search.best_params_
for param, value in best_params.items():
    print(f"  {param}: {value}")
print(f"\nEn iyi CV skoru (ROC-AUC): {random_search.best_score_:.4f} ({random_search.best_score_*100:.2f}%)")


## 🚀 7. Final Model Eğitimi

En iyi hiperparametrelerle final modeli eğitiyoruz. Bu model, tüm eğitim seti üzerinde eğitilecek.


In [ ]:
# Final model (en iyi parametrelerle)
final_model = XGBClassifier(
    **best_params,  # En iyi parametreleri kullan
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=-1,
    verbosity=0
)

print("Final model eğitiliyor...")
final_model.fit(X_train, y_train)
print("Eğitim tamamlandı!")


## 📊 8. Model Değerlendirme

Model performansını test seti üzerinde değerlendiriyoruz. Aşağıdaki metrikleri hesaplıyoruz:

### Kullanılan Metrikler:
1. **Accuracy**: Doğru tahmin oranı
2. **ROC-AUC**: Sınıflandırma kalitesi (0.5 = rastgele, 1.0 = mükemmel)
3. **Precision**: Pozitif tahminlerin doğruluğu
4. **Recall**: Gerçek pozitiflerin ne kadarını yakaladığımız
5. **F1-Score**: Precision ve Recall'un harmonik ortalaması

Ayrıca **5-fold Cross Validation** ile modelin kararlılığını test ediyoruz.


In [ ]:
# Test set tahminleri
y_pred = final_model.predict(X_test)  # Sınıf tahminleri (0 veya 1)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]  # Olasılık tahminleri (0-1 arası)

# Metrikler
test_accuracy = accuracy_score(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_pred_proba)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

print("=" * 80)
print("MODEL PERFORMANS METRİKLERİ (TEST SET)")
print("=" * 80)
print(f"Accuracy:        {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"ROC-AUC:         {test_roc_auc:.4f} ({test_roc_auc*100:.2f}%)")
print(f"Precision:       {test_precision:.4f} ({test_precision*100:.2f}%)")
print(f"Recall:          {test_recall:.4f} ({test_recall*100:.2f}%)")
print(f"F1-Score:        {test_f1:.4f} ({test_f1*100:.2f}%)")
print()


In [ ]:
# Cross-validation skorları (model kararlılığını test etmek için)
print("Cross-Validation Skorları (5-fold):")
cv_scores = cross_val_score(final_model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"  ROC-AUC Ortalama: {cv_scores.mean():.4f} ({cv_scores.mean()*100:.2f}%)")
print(f"  ROC-AUC Std Sapma: {cv_scores.std():.4f} ({cv_scores.std()*100:.2f}%)")
print(f"  ROC-AUC Min: {cv_scores.min():.4f} ({cv_scores.min()*100:.2f}%)")
print(f"  ROC-AUC Max: {cv_scores.max():.4f} ({cv_scores.max()*100:.2f}%)")
print()


In [ ]:
# Classification Report (detaylı metrikler)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Ödendi', 'Temerrüt']))


## 💾 9. Model Kaydetme

Eğitilmiş modeli, preprocessor'ı ve ilgili bilgileri bir paket halinde kaydediyoruz. Bu paket, daha sonra modeli yükleyip kullanmak için gereklidir.

**Kaydedilen Dosya**: `credit_risk_model_20fold.pkl`

**İçerik**:
- Eğitilmiş model
- Preprocessor (veri ön işleme pipeline'ı)
- Özellik isimleri
- En iyi hiperparametreler
- Test set performans metrikleri


In [ ]:
# Model ve preprocessor'ı birlikte kaydet
model_package = {
    'model': final_model,
    'preprocessor': preprocessor,
    'feature_names': feature_names,
    'best_params': best_params,
    'test_accuracy': test_accuracy,
    'test_roc_auc': test_roc_auc
}

output_file = 'credit_risk_model_20fold.pkl'
joblib.dump(model_package, output_file)
print(f"✅ Model başarıyla kaydedildi: {output_file}")
print(f"\nKaydedilen bilgiler:")
print(f"  - Model: XGBoost Classifier")
print(f"  - Preprocessor: ColumnTransformer")
print(f"  - Özellik sayısı: {len(feature_names)}")
print(f"  - Test Accuracy: {test_accuracy:.4f}")
print(f"  - Test ROC-AUC: {test_roc_auc:.4f}")

print("\n" + "=" * 80)
print("🎉 EĞİTİM TAMAMLANDI!")
print("=" * 80)
